# Pointcloud Tree Analysis
An experimental notebook

We will compare and analyse isolated trees using three different point cloud sources:
1. _[AHN](https://www.ahn.nl) data_ (publicly available)
2. _Cyclo Media_
3. _Sonarski_

A visual impression of what the an isolated three looks like in the three datasets. 
This particular tree is located at 1219.13, 4874.34 (LTR: AHN, CycloMedia, Sonarski):

![Comparison of datasets (side-view)](../imgs/20_side.png)
![Comparison of datasets (top-view)](../imgs/20_top.png)

#### Import Modules

In [ ]:
# Uncomment to load the local package rather than the pip-installed version.
# Add project src to path.
import set_path

In [ ]:
# Import modules.
import numpy as np
import open3d as o3d

import src.utils.math_utils as math_utils
import src.utils.plot_utils as plot_utils
import src.utils.las_utils as las_utils

#### Load datasets

In [ ]:
ahn_tree_path = './datasets/single_selection/single_121913_487434_AHN.las'
cyclo_tree_path = './datasets/single_selection/single_121913_487434_Cyclo.las'
sonarski_tree_path = './datasets/single_selection/single_121913_487434_Sonarski.las'

In [ ]:
ahn_las = las_utils.read_las(ahn_tree_path)
cyclo_las = las_utils.read_las(cyclo_tree_path)
sonarski_las = las_utils.read_las(sonarski_tree_path)

## 1. Dataset statistics

In [ ]:
def cloud_statistics(las):
    n = las...
    x_range = las.headers.offset...
    y_range = las.headers.offset...
    z_range = las.headers.offset...
    avg_density = 

    print('hi :)')

In [ ]:
for tree_pcd in tree_pcds:
    las_utils.cloud_statistics(tree_pcd)